In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Load pre-trained UNet model
model = torch.load('path_to_pretrained_unet_model.pth')
model.eval()

# Define transformation to preprocess input images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Define function to apply mask to input image
def apply_mask(image, mask):
    mask = np.array(mask)
    masked_image = np.array(image)
    masked_image[mask == 0] = 0
    return masked_image

# Load and preprocess input image
input_image = Image.open('path_to_input_image.jpg').convert('RGB')
input_tensor = transform(input_image).unsqueeze(0)

# Perform inference
with torch.no_grad():
    output = model(input_tensor)

# Convert output to binary mask
mask = (output > 0.5).squeeze().numpy().astype(np.uint8)

# Apply mask to input image
masked_image = apply_mask(input_image, mask)

# Save masked image
masked_image = Image.fromarray(masked_image)
masked_image.save('masked_output.jpg')
